$X = [x^1 ... x^N]$ --> N initial datapoints of size d
we want instead
$Z = [z^1 ... z^N]$ ---> N compressed datapoints of size k<<d


PCA uses the TRUNCATED SVD (Single Value Decomposition) where: $A_k = \sum_1^k u_i σ_i v_i^T$

We use this kowledge to build a PROJECTION MATRIX, that is simply $U_k^T$ --> $Z = U_k^T X$





# PCA

    1. Take dataset $X$
    2. Compute the centroid version $X_c = X - c(X)$ where c(X) is the centroid corresponding to X (mean of coordinates along every direction)
    3. Compute the SVD of $X_c$
    4. Find $U_k$ given k
    5. Compute the new dataset: $Z = U_k^T X$

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv("data.csv")

print("Shape of data {}".format(data.shape))

print(data.head())

In [ ]:
# Convert into array
data = np.array(data)

# Split into samples and labels (X and Y)

X = data[:, 1:]
X = X.T

Y = data[:, 0]

print(X.shape, Y.shape)

d, N = X.shape

In [ ]:
import matplotlib.pyplot as plt

def visualize(X, idx):
    img = X[:, idx]

    img = np.reshape(img, (28, 28))

    plt.imshow(img, cmap="gray")
    plt.show()

idx = 10
visualize(X, idx)

print("The associated digit is {}".format(Y[idx]))

In [ ]:
def split_data(X, Y, Ntrain):
    d, N = X.shape

    idx = np.arange(N)
    np.random.shuffle(idx)

    train_idx = idx[:Ntrain]
    test_idx = idx[Ntrain:]

    Xtrain = X[:, train_idx] # slicing with index array 
    Ytrain = Y[train_idx]
    
    Xtest = X[:, test_idx]
    Ytest = Y[test_idx]

    return (Xtrain, Ytrain), (Xtest, Ytest)

# Test it
(Xtrain, Ytrain), (Xtest, Ytest) = split_data(X, Y, 30_000)

print(Xtrain.shape, Xtest.shape)

### SVD of Matrix A

```
U, S, VT = np.linalg.svd(A, full_matrices=False)
```
And remember that to take only the first k, you simply impose the last (n-k columns of U to zeros!)


# Working example to plot clusters
THE LABLES HAVE TO BE IN Y as different values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create two cloud of points in 2-dimensional space
# and plot it, divided by color
x1 = np.random.normal(0, 1, (2, 1000))
x2 = np.random.normal(3, 0.1, (2, 100))

y1 = np.zeros((1000, ))
y2 = np.ones((100, ))

# Visualize them (coloring by class)

# Join together x1 x2, y1 y2
X = np.concatenate((x1, x2), axis=1)
Y = np.concatenate((y1, y2))

# Visualize
plt.scatter(X[0, :], X[1, :], c=Y)
plt.show()

# Linear Discriminant Analysis

This is a Supervised Version of PCA, that shifts each column by THE CENTROID OF ITS CLASS instead of the whole dataset's centroid

Define K classes and define $I_k = [i_1, ..., i_{N_k}]$ as the VECTOR of INDECES of all the samples in class k (for simplicity we order the initial dataset by class)

We define the centroid:
\begin{align}
    $c_k(X) = \frac{1}{N_k} \sum_{i \in I_k} x^i $
\end{align}

and the global centroid the usual way

### Within-cluster
CENTERED MATRIX becomes $X_{k,c} = X_k - c_k(X)$

Concatenating them all we obtain $ X_w = [X_{1,c} ... X_{K,c}] $

THE WITHIN-CLUSTER SCATTER MATRIX is $S_w = X_w X_w^T$ that represents the correlation matrix for points INSIDE each cluster

### Between-cluster
instead if you just concatenate the $k-th$ centroid $N_k$ times, and then concatenate all of these $K$ matrices together, you have a matrix with repeated centroids (each centroid counted as many times as samples clustered to it) and then shift all by the global centroid --> BETWEEN CLUSTERS SCATTERING MATRIX
$S_b = X_{strange} X_{strange}^T$




LDA: within-clusters distance of the projected data is as small as possible, while the between-clusters distance of the projected data is as big as possible

We want k ORTHONORMAL vectors $q_i$ that create a matrix $Q$, and MAXIMIZE the function $H(q) = \frac{q^T S_w q}{q^T S_b q}$

BUT $H(\alpha q) = H(q) $ if $\alpha >0$ --> simply FIX $q^T S_b q=1$ and find $\max_q q^T S_w q$


The implementation simply follows what we just described.

Given the input data and the labels:

1. Compute $S_w$ and $S_w$ as described above
2. If possible, compute the Cholesky decomposition of $S_w = L L^T$
3. If not, compute the Cholesky decomposition of $S_w + ϵI = L L^T$ (ϵ ~ 1e-6)
4. Find the first $k$ eigenvectors of $L^{-1} S_b L$ and collect them in a matrix $W$
5. Compute $Q = L^{-T} W$ and $Q^T$

Now $Z = Q^T X$

In [ ]:
data = pd.read_csv('data.csv')

# Convert data into a matrix
data = np.array(data)

X = data[:, 1:]
X = X.T
Y = data[:, 0]

d, N = X.shape

# Find the corresponding indeces
I1 = (Y==0)
I2 = (Y==6)
I3 = (Y==9)

# Split X and Y into X1, X2, X3 and Y1, Y2, Y3
X1 = X[:, I1]
X2 = X[:, I2]
X3 = X[:, I3]

Y1 = Y[I1]
Y2 = Y[I2]
Y3 = Y[I3]

# Concatenate the data
X = np.concatenate((X1, X2, X3), axis=1)
Y = np.concatenate((Y1, Y2, Y3))

In [ ]:
# Within-clusters centroid
C1 = np.mean(X1, axis=1)
C2 = np.mean(X2, axis=1)
C3 = np.mean(X3, axis=1)

# Global centroid
C = np.mean(X, axis=1)


In [ ]:
# Center each cluster dataset
X1c = X1 - C1.reshape((d, 1))
X2c = X2 - C2.reshape((d, 1))
X3c = X3 - C3.reshape((d, 1))

# Compute the within-cluster matrix by concatenation
Xw = np.concatenate((X1c, X2c, X3c), axis=1)

# Compute the within-cluster scatter matrix
Sw = Xw @ Xw.T

In [ ]:
# Compute the Xbars
Xbar1 = np.repeat(C1.reshape(d, 1), X1.shape[1], axis=1)
Xbar2 = np.repeat(C2.reshape(d, 1), X2.shape[1], axis=1)
Xbar3 = np.repeat(C3.reshape(d, 1), X3.shape[1], axis=1)

# Compute the between-cluster dataset
Xbar = np.concatenate((Xbar1, Xbar2, Xbar3), axis=1)

# Compute the between-cluster centered dataset
Xbarc = Xbar - C.reshape((d, 1))

# Compute the between-cluster scatter matrix
Sb = Xbarc @ Xbarc.T

In [ ]:
# We want to compute the Cholesky decomposition of Sw
try:
    L = np.linalg.cholesky(Sw)
except:
    epsilon = 1e-6
    Sw = Sw + epsilon * np.eye(Sw.shape[0])

    L = np.linalg.cholesky(Sw)

In [ ]:
import scipy
import scipy.sparse
import scipy.sparse.linalg

k = 2

# Compute the first k eigenvector decomposition of L^-1 Sb L
_, W = scipy.sparse.linalg.eigs(np.linalg.inv(L) @ Sb @ L, k=k)
W = np.real(W) # should be real but become complex for approximations --> bring them back to real

# Compute Q
Q = np.linalg.inv(L).T @ W

In [ ]:
# Compute the projection
Z = Q.T @ X

# Classification with clustering algorithms

1. Compute the projection matrix
2. In the PROJECTED SPACE, compute the centroids
3. For a NEW datapoint, project it $z = Px$ and classify it with the nearest centroid

